In [19]:
import pandas as pd

# Example of loading datasets (assuming they are in CSV format)
AAPL = pd.read_csv('Filtered_Data/AAPL_filtered.csv', index_col='Local Date')
AMZN = pd.read_csv('Filtered_Data/AMZN_filtered.csv', index_col='Local Date')
ARCA_TECH = pd.read_csv('Filtered_Data/ARCA_TECH_filtered.csv', index_col='Local Date')
BIT_USD = pd.read_csv('Filtered_Data/BIT_USD_filtered.csv', index_col='Local Date')
GOOGL = pd.read_csv('Filtered_Data/GOOGL_filtered.csv', index_col='Local Date')
MSFT = pd.read_csv('Filtered_Data/MSFT_filtered.csv', index_col='Local Date')
NASDAQ = pd.read_csv('Filtered_Data/NASDAQ_filtered.csv', index_col='Local Date')
OIL = pd.read_csv('Filtered_Data/OIL_filtered.csv', index_col='Local Date')
TSLA = pd.read_csv('Filtered_Data/TSLA_filtered.csv', index_col='Local Date')

AAPL = AAPL[['AAPL_Log_Change','AAPL_High_Low_Spread','AAPL_Volume','AAPL_1hr_Moving_Average','AAPL_RSI']]
AMZN = AMZN[['AMZN_Log_Change','AMZN_High_Low_Spread','AMZN_Volume','AMZN_1hr_Moving_Average','AMZN_RSI']]
ARCA_TECH = ARCA_TECH[['ARCA_TECH_Log_Change','ARCA_TECH_1hr_Moving_Average']]
BIT_USD = BIT_USD[['BTC_Log_Change','BTC_Bid_ask_spread']]
GOOGLE = GOOGL[['GOOGL_Log_Change','GOOGL_High_Low_Spread','GOOGL_Volume','GOOGL_1hr_Moving_Average','GOOGL_RSI']]
MSFT = MSFT[['MSFT_Log_Change','MSFT_High_Low_Spread','MSFT_Volume','MSFT_1hr_Moving_Average','MSFT_RSI']]
NASDAQ = NASDAQ[['NASDAQ_Log_Change','NASDAQ_1hr_Moving_Average']]
OIL = OIL[['OIL_Log_Change','OIL_Volume','OIL_High_Low_Spread']]
TSLA = TSLA[['TSLA_Log_Change','TSLA_High_Low_Spread','TSLA_Volume','TSLA_1hr_Moving_Average','TSLA_RSI']]


# Joining datasets on the index of dataset1
combined_dataset = AAPL.join([AMZN, ARCA_TECH, BIT_USD, GOOGL, MSFT, NASDAQ, OIL, TSLA], how='left')
na_row_indices = combined_dataset[combined_dataset.isna().any(axis=1)].index

index_list = na_row_indices.tolist()




In [20]:
#Fill in missing values
combined_dataset = combined_dataset[19:-1]
combined_dataset.shape
combined_dataset.to_csv('combined_dataset.csv')

In [22]:
import pandas as pd
import numpy as np

def fill_with_closest_average(df):
    for column in df.columns:
        for i in df[df[column].isna()].index:
            # Convert index positions to integer locations
            idx_position = df.index.get_loc(i)

            # Find the index of the next non-missing value
            next_valid_index = df[column].iloc[idx_position:].dropna().first_valid_index()
            prev_valid_index = df[column].iloc[:idx_position].dropna().last_valid_index()

            # Get the next and previous non-missing values
            next_valid = df.at[next_valid_index, column] if next_valid_index is not None else np.nan
            prev_valid = df.at[prev_valid_index, column] if prev_valid_index is not None else np.nan

            # Compute the average if both values are non-missing
            if not pd.isna(next_valid) and not pd.isna(prev_valid):
                df.at[i, column] = (next_valid + prev_valid) / 2
            elif not pd.isna(next_valid):
                df.at[i, column] = next_valid
            elif not pd.isna(prev_valid):
                df.at[i, column] = prev_valid

    return df

# Apply the function to your DataFrame
combined_dataset_filled = fill_with_closest_average(combined_dataset)

# Check the results
print(combined_dataset_filled)


                     AAPL_Log_Change  AAPL_High_Low_Spread  AAPL_Volume  \
Local Date                                                                
2023-11-08 14:25:00        -0.000230                 0.088     227644.0   
2023-11-08 14:20:00         0.000230                 0.130     277715.0   
2023-11-08 14:15:00        -0.000027                 0.080     231994.0   
2023-11-08 14:10:00         0.000082                 0.140     279311.0   
2023-11-08 14:05:00        -0.000612                 0.230     287958.0   
...                              ...                   ...          ...   
2022-11-21 09:55:00        -0.001078                 0.530    1055360.0   
2022-11-21 09:50:00         0.000209                 0.740    1204018.0   
2022-11-21 09:45:00        -0.003819                 0.600    1301872.0   
2022-11-21 09:40:00        -0.001083                 1.660    1580060.0   
2022-11-21 09:35:00         0.002559                 1.490    2588802.0   

                     AAP